In [ ]:
import bk.load
import bk.plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import neuroseries as nts

from scipy.stats import spearmanr,pearsonr, wilcoxon, zscore, ttest_ind, linregress

import os
import re
from itertools import product


%matplotlib qt
import pl


In [ ]:
# importing the directories of data for each session
paths = pd.read_csv('Z:/All-Rats/Billel/session_indexing.csv',sep = ';')['Path']

In [ ]:
# for z-scored FR
all_ES_z=pl.norm_FR_ES(paths, region='Hpc', celltype='Pyr', min=0, max=2040, bin=60, min_dur=30, pre_post=False)

In [ ]:
# for raw FR 
all_ES=pl.pop_FR_ES(paths=paths, region='BLA', celltype='Int', bin=60, min_dur=30, pre_post=False)

In [ ]:
# visualizing how FR looks like
for i in range(len(all_ES)):
    plt.scatter(np.arange(len(all_ES[i])),all_ES[i])
plt.title('Firing rate of each ES')
plt.ylabel('Firing Rate')
plt.xlabel('Time (m)')

In [ ]:
# regression line within each ES
coefficient=[]
plt.figure()
for i in range(len(all_ES)):
    fr_t=list(zip(all_ES[i], np.arange(len(all_ES[i])))) # zipping together x (time) and y (pop firing rate) values

    a=np.array(fr_t)

    # removing nan values
    filt = np.isfinite(a[:,0])
    y = a[filt,0]
    x = a[filt,1]
    
    
    if len(x)>0:

        # linear regression
        slope, intercept, r, p, se=linregress(x,y)
        coefficient.append((slope, intercept, r, p, se))
        
        
#         if slope>=0.5:
        
            # visualizing the linear regression fitted line
        plt.scatter(x,y)
        plt.plot(x, intercept + slope*x, 'b', label='fitted line')
        plt.title('Firing rate of BLA-INT during all ES')
        plt.ylabel('Firing Rate')
        plt.xlabel('Time (m)')

coefficient=np.array(coefficient)
np.mean(coefficient[:,2])

In [ ]:
# regression line for all ES pooled together

fr_t=[]
for i in range(len(all_ES)):
    fr_t.extend(list(zip(all_ES[i], np.arange(len(all_ES[i]))))) # zipping together x (time) and y (pop firing rate) values
    
a=np.array(fr_t)

# removing nan values
filt = np.isfinite(a[:,0])
y = a[filt,0]
x = a[filt,1]

# linear regression
slope, intercept, r, p, se=linregress(x,y)

# visualizing the linear regression fitted line
plt.scatter(x,y)
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.title('Firing rate for all ES')
plt.ylabel('Firing Rate (z)')
plt.xlabel('Time (s)')

In [ ]:
# regression line within each ES stage

all_ES_rem=pop_FR_ES_state(state='rem')


slopes=[]
plt.figure()
for i in range(len(all_ES_rem)):
    fr_t=list(zip(all_ES_rem[i], np.arange(len(all_ES_rem[i])))) # zipping together x (time) and y (pop firing rate) values

    a=np.array(fr_t)

    # removing nan values
    if len(a)>0:
        filt = np.isfinite(a[:,0])
        y = a[filt,0]
        x = a[filt,1]

        if len(x)>0:

            # linear regression
            slope, intercept, r, p, se=linregress(x,y)
            slopes.append((slope, intercept, r, p, se))
            
#             if slope>0.1:

            # visualizing the linear regression fitted line
            plt.scatter(x,y)
            plt.plot(x, intercept + slope*x, 'b', label='fitted line')
            plt.title('Firing rate of all ES')
            plt.ylabel('Firing Rate')
            plt.xlabel('Time (m)')

slopes=np.array(slopes)
np.nanmean(slopes[:,0])

# Statistical Comparisons

Normalizing the firing rates and then taking average of FR in a fixed number of bins (so all ES have the same length)

In [ ]:
all_ES_norm_pre,all_ES_norm_post =pl.pop_FR_ES_norm(paths=paths, region='Hpc', celltype='Pyr', bin=1, min_dur=30, pre_post=True)

In [ ]:
plt.figure()
plt.title("Pre-RUN FR for PYR-HPC across ES sessions")
plt.ylabel("Normalized FR")
plt.xlabel("Time Bins")
for i in range(len(all_ES_norm_pre)):
    mean, bin_edges, count=binned_statistic(np.arange(len(all_ES_norm_pre[i][0])),all_ES_norm_pre[i][0], bins=40)
    plt.scatter(np.arange(len(mean)),mean, 
           label='binned normalized FR')
plt.legend()

# Trends in successive NREM/REM epochs

In [ ]:
# regression line within each ES
def regression_graph(paths=paths, region='Hpc', celltype='Pyr', state='nrem', bin=1, min_dur=30, pre_post=False):
    """
    -> generates graph including average firing rate for each successive REM or NREM period in each ES and the associated line 
    -> returns array with slope, intercept, r, p, and se for each ES
    """
    data=pl.pop_FR_ES_state(paths, region, celltype, state, bin, min_dur, pre_post)
    
    stats=[]
    plt.figure()
    for i in range(len(data)):
        fr_t=list(zip(data[i], np.arange(len(data[i])))) # zipping together x (time) and y (pop firing rate) values

        a=np.array(fr_t)

        # removing nan values
        if len(a)>0:
            filt = np.isfinite(a[:,0])
            y = a[filt,0]
            x = a[filt,1]

            if len(x)>0:

                # linear regression
                slope, intercept, r, p, se=linregress(x,y)
                stats.append((slope, intercept, r, p, se))


                # visualizing the linear regression fitted line
                plt.scatter(x,y)
                plt.plot(x, intercept + slope*x, 'b', label='fitted line')
                plt.title(region+"-"+celltype+' FR in successive '+state+' epochs')
                plt.ylabel('Firing Rate (spk/sec)')
                plt.xlabel('Epoch Number')

    stats=np.array(stats)
    print(np.nanmean(stats[:,2]))
    return stats

In [ ]:
regions=["BLA", "Hpc"]
cells=["Pyr", "Int"]
sleep_stage=['rem', 'nrem']

comb=list(product(regions, cells, sleep_stage))

In [ ]:
stats=[]
for i in comb:
    x=regression_graph(paths=paths, region=i[0], celltype=i[1], state=i[2], bin=1, min_dur=30, pre_post=False)
    stats.append(x)

In [ ]:
# table for stats to import in my methodology-results doc
stats=np.array(stats)
stats_flat=pl.flatten(stats)
columns=["slope", "intercept","r", "p", "se"]
table=pd.DataFrame(data=stats_flat, columns=columns)
table